In [1]:
import os
import json
import pandas as pd

In [2]:
path = '../results/02_bayesian_optimization'

# 1. Bayes Optimization Results

In [3]:
def results_parser(path):

    results_df = pd.DataFrame()

    for rd in os.listdir(path):

        curr_path = os.path.join(path, rd)

        # filtrando apenas os diretórios
        trials = [
            trial 
            for trial in os.listdir(curr_path) 
            if '.json' not in trial
        ]

        # abrindo os resultados de cada tentativa
        for trial in trials:
            curr_trial = os.path.join(curr_path, trial)

            file = os.path.join(curr_path, trial, [
                f
                for f in os.listdir(curr_trial)
                if '.json' in f
            ][0])

            with open(file, 'r') as f:
                curr_result = json.load(f)

            result = curr_result['hyperparameters']['values']
            result['epochs'] = curr_result['best_step']
            result['loss'] = curr_result['score']
            result['batch_size'] = int(rd.split('_')[-1])
            result['trial'] = int(trial.split('_')[1])

            results_df = pd.concat([
                results_df,
                pd.DataFrame(result, index=[0])
            ], ignore_index=True)
    
    return results_df.sort_values('loss')

## 1.1 Pretrain BERT

### 1.1.1 Book Reviews

In [4]:
# top 10 melhores resultados (bert com book reviews)
results_lit_review = results_parser(f'{path}/bert/book_reviews')
results_lit_review.head(10)

,hidden_size,num_hidden_layers,num_attention_heads,intermediate_size,hidden_dropout_prob,learning_rate,epochs,loss,batch_size,trial
18,768,6,4,1024,0.1,0.000067,5,6.372961,8,11
0,512,8,16,2048,0.2,0.000128,5,6.497189,8,15
19,512,10,16,2048,0.1,0.000075,5,6.511960,8,16
57,768,6,4,3072,0.1,0.000124,5,6.531558,16,4
4,768,8,12,2048,0.2,0.000048,5,6.532318,8,0
46,768,8,4,3072,0.1,0.000092,5,6.551585,16,13
12,256,12,4,1024,0.2,0.000120,5,6.638262,8,17
20,512,8,16,2048,0.3,0.000187,5,6.692791,32,15
56,512,6,16,3072,0.4,0.000149,5,6.718224,16,6
3,768,6,12,1024,0.5,0.000041,6,6.751915,8,10


In [5]:
# média de resultado por batch_size e arquitetura (bert com book reviews)
results_lit_review.groupby(
    ['batch_size']
).agg(
    {'loss': ['mean', 'std', 'median']}
).sort_values(['batch_size'])

loss                    
                mean       std    median
batch_size                              
8           6.862437  0.252776  6.936142
16          6.984916  0.204720  7.097733
32          7.015485  0.121421  7.053308

### 1.1.2 TV

In [6]:
# top 10 melhores resultados (bert com tv reviews)
results_tv_review = results_parser(f'{path}/bert/tv_reviews')
results_tv_review.head(10)

,hidden_size,num_hidden_layers,num_attention_heads,intermediate_size,hidden_dropout_prob,learning_rate,epochs,loss,batch_size,trial
34,768,6,12,1024,0.2,0.000102,9,3.492996,32,1
30,768,6,16,1024,0.4,0.000093,8,5.046766,32,14
10,512,10,4,2048,0.1,0.000040,7,5.071844,8,14
5,512,10,16,3072,0.1,0.000047,7,5.131644,8,9
45,768,10,12,2048,0.3,0.000038,7,5.411859,16,9
24,256,10,16,1024,0.2,0.000336,6,5.417662,32,0
58,512,12,16,1024,0.4,0.000094,6,5.459514,16,11
46,256,12,16,1024,0.4,0.000162,7,5.517826,16,13
4,768,6,4,1024,0.4,0.000041,5,5.533927,8,0
1,256,8,4,1024,0.2,0.000050,7,5.630235,8,3


In [7]:
# média de resultado por batch_size e arquitetura (bert com tv reviews)
results_tv_review.groupby(
    ['batch_size']
).agg(
    {'loss': ['mean', 'std', 'median']}
).sort_values(['batch_size'])

loss                    
                mean       std    median
batch_size                              
8           6.225119  0.648824  6.593739
16          6.325825  0.612352  6.773590
32          5.851689  0.902379  5.848976

## 1.2 Fine Tuning

In [8]:
def read_json(path: str) -> dict:
    """Reads a json file and returns a dictionary.
    
    Args:
        path: path to the json file

    Returns:
        dictionary with the contents of the json file
    """
    with open(path, "r") as f:
        return json.load(f)


def best_hyperparams(results: dict) -> dict:
    """Returns the best hyperparameters based on the F1 score.
    
    Args:
        results (dict): Dictionary with the results of the hyperparameter 
                        optimization.

    Returns:
        dict: Dictionary with the best hyperparameters.
    """
    best_values = {
        'trial': None,
        'F1': 0,
    }
    for trial, result in results.items():
        if result['value'] > best_values['F1']:
            best_values['trial'] = trial
            best_values['F1'] = result['value']

    print(f'Best trial: {best_values["trial"]}.')

    return results[best_values['trial']]

### 1.2.2 TV

In [9]:
# procurando qual tentativa apresentou o melhor desempenho
tv_results1 = read_json(f'{path}/fine_tuning/tv/baseline.json')
best_hyperparams(tv_results1)

Best trial: trial_5.


{'dropout_rate': 0.30000000000000004,
 'learning_rate': 4.356405490340622e-05,
 'value': 0.8242424242424243}

In [10]:
# procurando qual tentativa apresentou o melhor desempenho
tv_results2 = read_json(f'{path}/fine_tuning/tv/bert_visio.json')
best_hyperparams(tv_results2)

Best trial: trial_9.


{'dropout_rate': 0.5,
 'learning_rate': 0.00015039796790289732,
 'value': 0.7440944881889764}

In [11]:
# procurando qual tentativa apresentou o melhor desempenho
tv_results3 = read_json(f'{path}/fine_tuning/tv/d_bert_visio.json')
best_hyperparams(tv_results3)

Best trial: trial_10.


{'dropout_rate': 0.30000000000000004,
 'learning_rate': 2.0863364780543777e-05,
 'value': 0.8501026694045174}

In [12]:
# procurando qual tentativa apresentou o melhor desempenho
tv_results4 = read_json(f'{path}/fine_tuning/tv/d_bert_visio_ffn.json')
best_hyperparams(tv_results4)

Best trial: trial_13.


{'dropout_rate': 0.4,
 'first_layer_units': 768,
 'second_layer_units': 1536,
 'third_layer_units': None,
 'learning_rate': 2.001383160164505e-05,
 'value': 0.830188679245283}

In [18]:
# procurando qual tentativa apresentou o melhor desempenho
tv_results5 = read_json(f'{path}/fine_tuning/tv/lora_baseline.json')
best_trial = (None, 0.0)
for trial, infos in tv_results5.items():
    if 'metrics' in infos.keys():
        if infos['metrics']['f1'] > best_trial[1]:
            best_trial = (trial, infos['metrics']['f1'])
tv_results5[best_trial[0]]

{'hyperparameters': {'r': 22,
  'lora_alpha': 29,
  'lora_dropout': 0.3349449195238189,
  'bias': 'all',
  'learning_rate': 0.0009978611060488306,
  'batch_size': 8,
  'weight_decay': 0.06270649322978346,
  'epochs': 13},
 'info': {'trainable_params': 889350,
  'all_params': 109221894,
  'trainable%': 0.8142598223026603},
 'metrics': {'precision': 0.8464912280701754,
  'recall': 0.8427947598253275,
  'f1': 0.8446389496717724,
  'accuracy': 0.9734369938451571}}

### 1.2.3 ReLi

In [ ]:
# procurando qual tentativa apresentou o melhor desempenho
reli_results1 = read_json(f'{path}/fine_tuning/reli/baseline.json')
best_hyperparams(reli_results1)

Best trial: trial_9.


{'dropout_rate': 0.1, 'learning_rate': 2e-05, 'value': 0.602}

In [ ]:
# procurando qual tentativa apresentou o melhor desempenho
reli_results2 = read_json(f'{path}/fine_tuning/reli/litera_bert.json')
best_hyperparams(reli_results2)

Best trial: trial_8.


{'dropout_rate': 0.5,
 'learning_rate': 3.282787355738218e-05,
 'value': 0.4633333333333333}

In [ ]:
# procurando qual tentativa apresentou o melhor desempenho
reli_results3 = read_json(f'{path}/fine_tuning/reli/d_litera_bert.json')
best_hyperparams(reli_results3)

Best trial: trial_9.


{'dropout_raNonete': 0.5,
 'learning_rate': 2.6417102416521368e-05,
 'value': 0.6220735785953178}

In [ ]:
# procurando qual tentativa apresentou o melhor desempenho
reli_results4 = read_json(f'{path}/fine_tuning/reli/d_litera_bert_ffn.json')
best_hyperparams(reli_results4)

Best trial: trial_9.


{'dropout_rate': 0.0,
 'first_layer_units': 256,
 'second_layer_units': 512,
 'third_layer_units': 768,
 'learning_rate': 3.8534252343702257e-05,
 'value': 0.4692737430167598}

In [20]:
# procurando qual tentativa apresentou o melhor desempenho
reli_results5 = read_json(f'{path}/fine_tuning/reli/lora_baselie.json')
best_trial = (None, 0.0)
for trial, infos in reli_results5.items():
    if 'metrics' in infos.keys():
        if infos['metrics']['f1'] > best_trial[1]:
            best_trial = (trial, infos['metrics']['f1'])
reli_results5[best_trial[0]]

{'hyperparameters': {'r': 12,
  'lora_alpha': 27,
  'lora_dropout': 0.4564039673376695,
  'bias': 'none',
  'learning_rate': 0.0003086875775669831,
  'batch_size': 32,
  'weight_decay': 0.0855465837461197,
  'epochs': 7},
 'info': {'trainable_params': 446982,
  'all_params': 108779526,
  'trainable%': 0.41090636853850604},
 'metrics': {'precision': 0.678714859437751,
  'recall': 0.6035714285714285,
  'f1': 0.6389413988657844,
  'accuracy': 0.9875864514984927}}